In [9]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import time
import numpy as np

#Building FC NN

Model = torch.nn.Sequential(
            nn.Linear(784, 200),
            nn.ReLU(),
            nn.Linear(200,50),
            nn.ReLU(),
            nn.Linear(50,10)) 
#Note: Cross Entropy Loss from pytorch is used in this implementation 
#so there is no need to explicitly define a softmax layer
 
#Training

def train(model, device, train_loader, optimizer, loss, epoch):
    model.train()
    
    for batch_idx, (data, target) in enumerate(train_loader):
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        data = data.view(-1,784)
        output = model(data)
        if loss =="CE":
            loss_fn = nn.CrossEntropyLoss()
        
        loss_ = loss_fn(output, target)
        loss_.backward()
        optimizer.step()
        if batch_idx % 10 == 0:
            print('Train Epoch: {} [{}/{} ({:.0f}%)]\tLoss:{:.6f}'.format(
            epoch, batch_idx * len(data), len(train_loader.dataset), 
            100. * batch_idx/len(train_loader), loss_.item()))
            
def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            data = data.view(-1,784)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()

            pred = output.argmax(dim=1, keepdim=True)
            correct += pred.eq(target.view_as(pred)).sum().item()
        test_loss /= len(test_loader.dataset)

        print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%\n'.format(
            test_loss, correct, len(test_loader.dataset),
            100. * correct/len(test_loader.dataset)))
def main():
    no_cuda = True
    use_cuda = not no_cuda and torch.coda.is_available()
    device = torch.device("cuda" if use_cuda else "cpu")
    
    #loading MNIST dataset
    batch_size = 64
    test_batch_size = 10000
    train_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train = True, download = True,
                        transform=transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize((0.1307,), (0.3081))
                            ])),
        batch_size = batch_size, shuffle=True)
    test_loader = torch.utils.data.DataLoader(
        datasets.MNIST('../data', train=False, transform=transforms.Compose([
                            transforms.ToTensor(),
                            transforms.Normalize((0.1307,),(0.3081))
                        ])),
        batch_size = test_batch_size, shuffle=True)


    #setting optimizer

    lr = 0.01
    model = Model.to(device)
    optimizer = optim.SGD(model.parameters(), lr=lr)
    time0 = time.time()

    #Training Settings

    epochs = 10
    loss = 'CE'

    #Starting Training

    time0 = time.time()
    for epoch in range(1, epochs + 1):
        train(model, device, train_loader, optimizer, loss, epoch)
        test(model, device, test_loader)
    time1 = time.time()
    print('Training and Testing total execution time is: %s seconds' % (time1-time0))
if __name__ == '__main__':
    main()
        

Train Epoch: 1 [0/60000 (0%)]	Loss:2.307997
Train Epoch: 1 [640/60000 (1%)]	Loss:2.272585
Train Epoch: 1 [1280/60000 (2%)]	Loss:2.243087
Train Epoch: 1 [1920/60000 (3%)]	Loss:2.206149
Train Epoch: 1 [2560/60000 (4%)]	Loss:2.199751
Train Epoch: 1 [3200/60000 (5%)]	Loss:2.123604
Train Epoch: 1 [3840/60000 (6%)]	Loss:2.117667
Train Epoch: 1 [4480/60000 (7%)]	Loss:2.041716
Train Epoch: 1 [5120/60000 (9%)]	Loss:1.974222
Train Epoch: 1 [5760/60000 (10%)]	Loss:1.956349
Train Epoch: 1 [6400/60000 (11%)]	Loss:1.879450
Train Epoch: 1 [7040/60000 (12%)]	Loss:1.824574
Train Epoch: 1 [7680/60000 (13%)]	Loss:1.737759
Train Epoch: 1 [8320/60000 (14%)]	Loss:1.847108
Train Epoch: 1 [8960/60000 (15%)]	Loss:1.686346
Train Epoch: 1 [9600/60000 (16%)]	Loss:1.463709
Train Epoch: 1 [10240/60000 (17%)]	Loss:1.361266
Train Epoch: 1 [10880/60000 (18%)]	Loss:1.333445
Train Epoch: 1 [11520/60000 (19%)]	Loss:1.363361
Train Epoch: 1 [12160/60000 (20%)]	Loss:1.070390
Train Epoch: 1 [12800/60000 (21%)]	Loss:1.114287
